In [1]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import FastText 
from tqdm import tqdm

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
train_limpio = pd.read_csv('cleaned_train.csv')
test_limpio = pd.read_csv('cleaned_test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
id = '1eWJLOjsG04oOtYXJ6eta7VnudJPIlBh_'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('wiki-news-300d-1M-subword.vec')

In [5]:
def load_fasttext():
    
    print('loading word embeddings...')
    embeddings_index = {}
    f = open('wiki-news-300d-1M-subword.vec',encoding='utf-8')
    for line in tqdm(f):
        values = line.strip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))
    
    return embeddings_index

embeddings_index = load_fasttext()

1302it [00:00, 13011.02it/s]

loading word embeddings...


999995it [01:15, 13295.17it/s]

found 999995 word vectors


In [6]:
def get_average(text):
  counter = np.zeros(300)
  words = str(text).split(" ")
  for word in words:
    if word in embeddings_index.keys():
      counter += embeddings_index[word]
  return counter/len(words)

Elimino los features menos importantes segun el xgboost classifier

In [7]:
train_limpio.drop(columns = ['Unnamed: 0', 'Account_Created_Date', 'Month', 'Delivery_Quarter',
                             'Last_Modified_Year', 'Pricing, Delivery_Terms_Quote_Appr', 
                             'Delivery_Year', 'Planned_Delivery_End_Year', 'Region', 'Bureaucratic_Code_0_Approval', 
                             'Bureaucratic_Code_0_Approved', 'Quote_Expiry_Year', 
                             'Items_Count', 'Total_Taxable_Amount_Currency', 'Opportunity_Type', 
                             'Currency'], inplace=True)

In [8]:
train_limpio['Territory'] = train_limpio['Territory'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Billing_Country'] = train_limpio['Billing_Country'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Account_Name'] = train_limpio['Account_Name'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Opportunity_Name'] = train_limpio['Opportunity_Name'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Account_Owner'] = train_limpio['Account_Owner'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Opportunity_Owner'] = train_limpio['Opportunity_Owner'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Account_Type'] = train_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Quote_Type'] = train_limpio['Quote_Type'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Delivery_Terms'] = train_limpio['Delivery_Terms'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Last_Modified_By'] = train_limpio['Last_Modified_By'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Product_Family'] = train_limpio['Product_Family'].apply(lambda x: np.mean(get_average(x)))
train_limpio['ASP_Currency'] = train_limpio['ASP_Currency'].apply(lambda x: np.mean(get_average(x)))
train_limpio['ASP_(converted)_Currency'] = train_limpio['ASP_(converted)_Currency'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Total_Amount_Currency'] = train_limpio['Total_Amount_Currency'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Prod_Category_A'] = train_limpio['Prod_Category_A'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Account_Type'] = train_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Account_Type'] = train_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Product_Name'] = train_limpio['Product_Name'].apply(lambda x: np.mean(get_average(x)))
train_limpio['Bureaucratic_Code'] = train_limpio['Bureaucratic_Code'].apply(lambda x: np.mean(get_average(x)))	
	


In [9]:
test_limpio['Territory'] = test_limpio['Territory'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Billing_Country'] = test_limpio['Billing_Country'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Account_Name'] = test_limpio['Account_Name'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Opportunity_Name'] = test_limpio['Opportunity_Name'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Account_Owner'] = test_limpio['Account_Owner'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Opportunity_Owner'] = test_limpio['Opportunity_Owner'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Account_Type'] = test_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Quote_Type'] = test_limpio['Quote_Type'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Delivery_Terms'] = test_limpio['Delivery_Terms'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Last_Modified_By'] = test_limpio['Last_Modified_By'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Product_Family'] = test_limpio['Product_Family'].apply(lambda x: np.mean(get_average(x)))
test_limpio['ASP_Currency'] = test_limpio['ASP_Currency'].apply(lambda x: np.mean(get_average(x)))
test_limpio['ASP_(converted)_Currency'] = test_limpio['ASP_(converted)_Currency'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Total_Amount_Currency'] = test_limpio['Total_Amount_Currency'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Prod_Category_A'] = test_limpio['Prod_Category_A'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Account_Type'] = test_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Account_Type'] = test_limpio['Account_Type'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Product_Name'] = test_limpio['Product_Name'].apply(lambda x: np.mean(get_average(x)))
test_limpio['Bureaucratic_Code'] = test_limpio['Bureaucratic_Code'].apply(lambda x: np.mean(get_average(x)))

In [10]:
train_limpio.to_csv('data/cleaned_train_FT.csv')
test_limpio.to_csv('data/cleaned_test_FT.csv')